In [16]:
from model import Net
import numpy as np
from batch_generator import GenerateImageBatch
import torch
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
writer = SummaryWriter()


def dice_loss(input, target):
    smooth = 1.
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()

    return (1 - ((2. * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth)))

def main():

    # Load data
    x = Image.open('./dcm.jpg')
    np.save('./'+ str('train_image.npy'), x)
    np.save('./'+ str('train_label.npy'), x)
    train_images = np.load('./train_image.npy')
    train_labels = np.load('./train_label.npy')

    validation_images = train_images
    validation_labels = train_labels

    # Create batch architecture
    batch_size = 1
    epochs = 50
    learning_rate = 0.00001 
    train_data = GenerateImageBatch(train_images, train_labels, batch_size)
    validation_data = GenerateImageBatch(validation_images, validation_labels, batch_size)

    # Create model object
    net = Net(batch_size)
    net.cuda()

    net = net.float()

    # Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    criterion = dice_loss

    # Begin training
    for epoch in range(1, epochs+1):
        net.train()
        train_loss = 0
        for i in range(int(train_data.img_count / batch_size)):
            # Forward pass: Compute predicted y by passing x to the model

            train_data.next_batch()
            inputs = train_data.inputs
            inputs.astype(np.float32)
            inputs = np.transpose(inputs, (0, 3, 1, 2))

            inputs = torch.from_numpy(inputs)

            outputs = torch.from_numpy(train_data.outputs.astype(np.float32))
            outputs = np.transpose(outputs.float(), (0, 3, 1, 2))

            inputs, outputs = inputs.cuda(), outputs.cuda()
            inputs = inputs.type(torch.float32)
            outputs = outputs.type(torch.float32)

            y_pred = net(inputs)
            y_pred = y_pred.cuda()

            # Compute and print loss
            loss = criterion(y_pred, outputs)

            writer.add_scalar("Loss/train", loss, epoch)
            print('Loss:')
            print(loss)

            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Calculate Loss
            train_loss += loss.item()

        net.eval()
        valid_loss = 0
        for i in range(int(validation_data.img_count / batch_size)):
            validation_data.next_batch()
            inputs = validation_data.inputs
            inputs = np.transpose(inputs, (0, 3, 1, 2))
            inputs.astype(np.float32)

            inputs = torch.from_numpy(inputs)

            outputs = torch.from_numpy(validation_data.outputs)
            outputs = np.transpose(outputs, (0, 3, 1, 2))

            inputs, outputs = inputs.cuda(), outputs.cuda()
            inputs = inputs.type(torch.float32)
            outputs = outputs.type(torch.float32)

            y_pred = net(inputs)

            # Compute and print loss
            loss = criterion(y_pred, outputs)

            writer.add_scalar("Loss/validate", loss, epoch)
            valid_loss += loss.item()

        print(f'Epoch {epoch} \t\t Training Loss: {train_loss / (int(train_data.img_count / batch_size))} \t\t Validation Loss: {valid_loss / (int(validation_data.img_count / batch_size))}')
        print(f'Validation Loss {valid_loss:.6f}) \t Saving The Model')
        # Saving State Dict
        train = train_loss / (int(train_data.img_count / batch_size))
        valid = valid_loss / (int(validation_data.img_count / batch_size))
        torch.save(net.state_dict(), '../train_results/epoch_' + str(epoch) + '_train-loss_' + str(train) + '_val_loss_' + str(valid) + '.pth')

    writer.flush()
    writer.close()


if __name__ == "__main__":

    main()


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.